In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import joblib

def kategoriser_tidspunkt(tid: str) -> str:
    hour = int(tid.split(":")[0])
    if 7 <= hour < 9:
        return "morgenrush 07:00-08:59"
    elif 9 <= hour < 15:
        return "formiddag 09:00-14:59"
    elif 15 <= hour < 18:
        return "ettermiddagrush 15:00-17:59"
    elif 18 <= hour < 24:
        return "kveld 18:00-23:59"
    else:
        return "natt 00:00-06:59"

def kategoriser_temperatur(temp: float) -> str:
    if temp < -5:
        return "kaldere enn - 5 grader"
    elif -5 <= temp <= 5:
        return "-5 t.o.m. 5 grader"
    else:
        return "varmere enn 5 grader"

def rens_kommune(tekst: str) -> str:
    # Fjerner kommune-ID og gjør til lowercase
    return tekst.split(" - ")[-1].strip().lower()


df2 = pd.read_csv("../data/processed/Ulykker_med_temp.csv", encoding="ISO-8859-1", sep=";")
df2.columns = df2.columns.str.strip()
df2["Kommune"] = df2["Kommune"].apply(rens_kommune)

melted2 = df2.melt(id_vars=["Kommune", "Måned", "År"], var_name="Temperaturkategori", value_name="Antall")
melted2 = melted2.dropna(subset=["Antall", "Kommune", "Måned", "År", "Temperaturkategori"])
melted2["Temperaturkategori"] = melted2["Temperaturkategori"].str.lower()

melted2["Risiko"] = pd.cut(melted2["Antall"], bins=[-1, 0, 2, float("inf")], labels=["lav", "middels", "høy"])

# Modell
X2 = melted2[["Kommune", "Måned", "År", "Temperaturkategori"]]
y2 = melted2["Risiko"]

pipeline_temp = Pipeline([
    ("prep", ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["Kommune", "Måned", "Temperaturkategori"]),
        ("num", "passthrough", ["År"])
    ])),
    ("clf", RandomForestClassifier(n_estimators=30, random_state=42))
])

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)
pipeline_temp.fit(X2_train, y2_train)
joblib.dump(pipeline_temp, "modell_temp_pipeline.pkl")

def prediker_temp(kommune, måned, år, temperatur_float):
    kat_temp = kategoriser_temperatur(temperatur_float)
    kommune_renset = rens_kommune(kommune)
    input_df = pd.DataFrame([[kommune_renset, måned, år, kat_temp]],
                            columns=["Kommune", "Måned", "År", "Temperaturkategori"])
    return pipeline_temp.predict(input_df)[0]



In [8]:
temperatur_kategori = kategoriser_temperatur(-3.2)
resultat_temp = prediker_tempmodell("Oslo", "Januar", 2023, temperatur_kategori)

In [9]:
resultat_temp

'lav'